<a href="https://colab.research.google.com/github/ShoyebKhan09/Action_Detection/blob/main/Object_Detection_tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

In [7]:
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

In [10]:
import imageio
from urllib import request

In [11]:
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

Found 400 labels.


In [13]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  return imageio.mimsave('./animation.gif', converted_images, fps=25) 

In [22]:
#! gdown --id 1bbvRGtPhCABWWq0cSOntDc54e4aF774V

In [16]:
sample_video = '/content/VID-20220909-WA0001.mp4'

In [17]:
video_file = load_video(sample_video)

In [18]:
video_file.shape

(139, 224, 224, 3)

In [19]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [20]:
def predict(sample_video):
  # Add a batch axis to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

In [25]:
from IPython.display import HTML
from base64 import b64encode

In [26]:
mp4 = open('/content/VID-20220909-WA0001.mp4','rb').read()

In [27]:
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [28]:
predict(video_file)

Top 5 actions:
  shooting basketball   : 82.51%
  playing basketball    : 16.34%
  dunking basketball    :  0.61%
  dribbling basketball  :  0.36%
  passing American football (not in game):  0.18%


In [29]:
## Object Detection 